In [1]:
import json
import re
import ollama

In [2]:
# Configuration
INPUT_FILE = 'output/agent2_ideas.json'
OUTPUT_FILE = 'output/agent6_impact.json'
AGENT_6 = 'qwen3:32b'

In [3]:
def load_ideas(path):
    """
    Load project ideas from JSON produced by Agent 2.
    """
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [4]:
def estimate_impact(idea: dict) -> dict:
    """
    Call the LLM to estimate three sub-scores, compute an average impact_score, and return summary.
    """
    prompt = f"""
You are Agent 6: an Impact Estimator for climate-change deep learning projects.

Project idea:
Title: {idea['title']}
Description: {idea['description']}
ID: {idea.get('idea_id', '')}

Your task is to estimate the potential impact of this project on three metrics, each on a scale of 0.0 to 1.0:
1. EMISSIONS_REDUCTION: How effectively will this project reduce greenhouse gas emissions?
2. POLICY_APPLICABILITY: How useful will this project be for climate policy decisions?
3. SOCIAL_BENEFIT: How much will this project benefit society and communities?

RESPONSE FORMAT:
You must use this exact format with these exact headings:

IDEA_ID: {idea.get('idea_id', '')}
EMISSIONS_REDUCTION: [value between 0.0 and 1.0]
POLICY_APPLICABILITY: [value between 0.0 and 1.0]
SOCIAL_BENEFIT: [value between 0.0 and 1.0]
SUMMARY: [2-3 sentences explaining your reasoning]
"""
    try:
        # First try the chat API
        response = ollama.chat(
            model=AGENT_6,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        text = response['message']['content'] if 'message' in response else str(response)
    except Exception as e:
        print(f"Chat API error: {e}, falling back to generate")
        try:
            # Fallback to generate API
            response = ollama.generate(model=AGENT_6, prompt=prompt)
            text = response['response'] if 'response' in response else str(response)
        except Exception as e2:
            print(f"Generate API error: {e2}")
            text = f"Error calling LLM: {e2}"

    # Save raw response for debugging
    with open(f"output/debug/debug_{idea.get('idea_id', 'unknown')}.txt", "w") as f:
        f.write(text)

    print(f"\nRaw response for {idea.get('idea_id', '')}:\n{text[:200]}...\n")

    # Parse the text-based response
    idea_id = idea.get('idea_id', '')  # Default to input idea_id
    emissions_reduction = 0
    policy_applicability = 0
    social_benefit = 0
    summary = ""

    # Clean up response
    text = text.replace('```', '').strip()

    # Extract metrics using regex as a more robust approach
    emissions_match = re.search(r"EMISSIONS_REDUCTION:?\s*([0-9]*\.?[0-9]+)", text, re.IGNORECASE)
    if emissions_match:
        try:
            emissions_reduction = float(emissions_match.group(1))
            print(f"Found emissions reduction: {emissions_reduction}")
        except ValueError:
            print(f"Error parsing emissions value: {emissions_match.group(1)}")

    policy_match = re.search(r"POLICY_APPLICABILITY:?\s*([0-9]*\.?[0-9]+)", text, re.IGNORECASE)
    if policy_match:
        try:
            policy_applicability = float(policy_match.group(1))
            print(f"Found policy applicability: {policy_applicability}")
        except ValueError:
            print(f"Error parsing policy value: {policy_match.group(1)}")

    social_match = re.search(r"SOCIAL_BENEFIT:?\s*([0-9]*\.?[0-9]+)", text, re.IGNORECASE)
    if social_match:
        try:
            social_benefit = float(social_match.group(1))
            print(f"Found social benefit: {social_benefit}")
        except ValueError:
            print(f"Error parsing social value: {social_match.group(1)}")

    # Extract summary
    summary_match = re.search(r"SUMMARY:?\s*(.+?)(?=\n\n|\n[A-Z_]+:|\Z)", text, re.DOTALL | re.IGNORECASE)
    if summary_match:
        summary = summary_match.group(1).strip()
        print(f"Found summary: {summary[:50]}...")

    # If we didn't find a summary, try to get any text after the last metric
    if not summary:
        last_metric_pos = max(
            text.lower().rfind("emissions_reduction:"),
            text.lower().rfind("policy_applicability:"),
            text.lower().rfind("social_benefit:")
        )
        if last_metric_pos != -1:
            # Find the end of the line
            line_end = text.find("\n", last_metric_pos)
            if line_end != -1:
                remaining_text = text[line_end:].strip()
                if remaining_text:
                    summary = remaining_text
                    print(f"Extracted summary from remaining text: {summary[:50]}...")

    # If still empty, look for any full sentences in the response
    if not summary:
        sentences = re.findall(r'[A-Z][^.!?]*[.!?]', text)
        if len(sentences) >= 2:
            summary = ' '.join(sentences[:2])
            print(f"Extracted the first two sentences as summary: {summary[:50]}...")

    # If we have zero scores but can find numbers in the text, use those as a fallback
    if emissions_reduction == 0 and policy_applicability == 0 and social_benefit == 0:
        print("All scores are zero, trying to extract any numbers as a fallback...")
        numbers = re.findall(r'([0-9]*\.?[0-9]+)', text)
        numbers = [float(n) for n in numbers if 0 <= float(n) <= 1]
        if len(numbers) >= 3:
            emissions_reduction = numbers[0]
            policy_applicability = numbers[1]
            social_benefit = numbers[2]
            print(f"Using extracted numbers: {emissions_reduction}, {policy_applicability}, {social_benefit}")

    # If we still have zeros, set default non-zero values
    if emissions_reduction == 0:
        emissions_reduction = 0.5
        print("Using default emissions_reduction value: 0.5")
    if policy_applicability == 0:
        policy_applicability = 0.5
        print("Using default policy_applicability value: 0.5")
    if social_benefit == 0:
        social_benefit = 0.5
        print("Using default social_benefit value: 0.5")

    # Calculate impact score
    scores = [emissions_reduction, policy_applicability, social_benefit]
    impact_score = sum(scores) / len(scores)

    print(f"Final impact score: {impact_score:.2f}")

    return {
        'idea_id': idea_id,
        'impact_score': impact_score,
        'summary': summary
    }

In [5]:
def run_agent6():
    ideas = load_ideas(INPUT_FILE)
    impacts = []

    print(f"Loaded {len(ideas)} ideas from {INPUT_FILE}")

    for i, idea in enumerate(ideas):
        print(f"\nProcessing idea {i+1}/{len(ideas)}: {idea['title']}")
        impact = estimate_impact(idea)
        impacts.append(impact)
        print(f"Impact score: {impact['impact_score']:.2f}")
        print(f"Summary: {impact['summary'][:100]}...")

    print(f"\nWriting {len(impacts)} impact assessments to {OUTPUT_FILE}")

    with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
        json.dump(impacts, f, ensure_ascii=False, indent=2)

    print("Done!")

if __name__ == "__main__":
    run_agent6()

Loaded 20 ideas from output/agent2_ideas.json

Processing idea 1/20: Arctic Anomaly Detection with Computer Vision

Raw response for 8_arctic_anomaly_detection_with_computer_vision:
<think>
Okay, let me try to work through this. The project is about using computer vision to detect Arctic climate anomalies like ice melt and temperature spikes. The user wants me to rate it on three...

Found emissions reduction: 0.3
Found policy applicability: 0.8
Found social benefit: 0.7
Found summary: should explain that the project's main impact is i...
Final impact score: 0.60
Impact score: 0.60
Summary: should explain that the project's main impact is in monitoring and informing policy and communities ...

Processing idea 2/20: Climate Resilience Prediction using GANs

Raw response for 9_climate_resilience_prediction_using_gans:
<think>
Okay, let's tackle this. The user wants me to estimate the impact of the project on three metrics: EMISSIONS_REDUCTION, POLICY_APPLICABILITY, and SOCIAL_BENEFIT. Th